In [271]:
import pandas as pd


In [272]:
url = (
    "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?"
    "table=cumulative&format=csv"
)

# Directly load into DataFrame
df = pd.read_csv(url)

print(df.head())

URLError: <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>

In [ ]:
print(df.shape)

(9564, 50)


In [ ]:
df.drop({'koi_kepmag_err','kepid', 'kepoi_name', 'koi_tce_delivname', 'koi_score', 'kepler_name', 'koi_pdisposition', 'koi_teq_err2', 'koi_teq_err1'}, axis=1, inplace=True)

In [ ]:
# Map to three categories
df['koi_disposition_class'] = df['koi_disposition'].apply(
    lambda x: 'CONFIRMED' if x == 'CONFIRMED' 
              else ('CANDIDATE' if x == 'CANDIDATE' else 'OTHER')
)

# Encode into numbers
class_map = {'CONFIRMED': 0, 'CANDIDATE': 1, 'OTHER': 2}
df['koi_disposition_encoded'] = df['koi_disposition_class'].map(class_map)

print(df[['koi_disposition', 'koi_disposition_class', 'koi_disposition_encoded']].head())


  koi_disposition koi_disposition_class  koi_disposition_encoded
0       CONFIRMED             CONFIRMED                        0
1       CONFIRMED             CONFIRMED                        0
2       CANDIDATE             CANDIDATE                        1
3  FALSE POSITIVE                 OTHER                        2
4       CONFIRMED             CONFIRMED                        0


In [ ]:
df = df.select_dtypes(exclude=['object'])

In [ ]:
df = df.fillna(value=np.nan)
print(df.shape)
df = df.dropna()
print(df.shape)

(9564, 39)
(8744, 39)


In [ ]:
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

print(df.shape)
corr_matrix = df.corr().abs()

# Select upper triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop columns with correlation > 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
df = df.drop(columns=to_drop)

print(df.shape)


(8744, 39)
(8744, 33)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import sklearn as sk
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.linear_model import LogisticRegression


print(df.shape)

vif = pd.DataFrame()
vif["feature"] = df.columns
vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

high_vif_cols = vif[vif['VIF'] > 10]['feature'].tolist()
high_vif_cols = [col for col in high_vif_cols if col != 'const']
df = df.drop(columns=high_vif_cols)

print(vif)



vif = pd.DataFrame()
vif["feature"] = df.columns
vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
print(vif)
print(df.shape)

(8744, 33)
                    feature         VIF
0             koi_fpflag_nt    1.003107
1             koi_fpflag_ss    2.545152
2             koi_fpflag_co    2.381516
3             koi_fpflag_ec    1.698439
4                koi_period    3.892792
5           koi_period_err1    2.452886
6               koi_time0bk   12.794900
7          koi_time0bk_err1    1.823890
8                koi_impact    5.178662
9           koi_impact_err1    1.199026
10          koi_impact_err2    3.246968
11             koi_duration    2.399314
12        koi_duration_err1    2.291116
13                koi_depth    2.036386
14           koi_depth_err1    1.103399
15                 koi_prad    2.439093
16            koi_prad_err1    3.090666
17                  koi_teq    7.797969
18                koi_insol    9.531210
19           koi_insol_err1    9.156558
20            koi_model_snr    2.011694
21         koi_tce_plnt_num    4.997112
22                koi_steff   85.819508
23           koi_steff_err1  

In [ ]:
# Multinomial logistic regression

X_train, X_test, y_train, y_test = train_test_split(df.drop("koi_disposition_encoded", axis = 1), df["koi_disposition_encoded"] , test_size = 0.2)
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)



clf = LogisticRegression(
    max_iter=1000,
    multi_class='multinomial',
    solver='lbfgs'
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", sk.metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.49514008004574045


c:\Users\santi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
xgb_model = XGBClassifier(
    objective='binary:logistic',
    random_state=42,) 
)

In [ ]:
# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

In [ ]:
print("Accuracy:", sk.metrics.accuracy_score(y_test, y_pred))